Demonstrating how to get DonkeyCar Tub files into a PyTorch/fastai DataBlock

In [ ]:
from fastai.data.all import *
from fastai.vision.all import *
from fastai.data.transforms import ColReader, Normalize, RandomSplitter
import torch

In [ ]:
from donkeycar.parts.tub_v2 import Tub
import pandas as pd
from pathlib import Path

In [ ]:
from malpi.dk.train import preprocessFileList

Use this cell if you have a file with a list of Tubs to load.
Use the next cell if you want to load all Tubs in a directory.

In [ ]:
tub_dirs = preprocessFileList(["track1_warehouse.txt"])
tubs = []
count = 0
root_path = Path("data")
for item in tub_dirs:
    if Path(item).is_dir():
        try:
            t = Tub(str(item),read_only=True)
        except FileNotFoundError as ex:
            continue
        except ValueError as ex:
            # In case the catalog file is empty
            continue
        tubs.append(t)
        count += len(t)
count

In [ ]:
tubs = []
root_path = Path("data")
for item in root_path.iterdir():
    if item.is_dir():
        try:
            t = Tub(str(item),read_only=True)
        except FileNotFoundError as ex:
            continue
        except ValueError as ex:
            # In case the catalog file is empty
            continue
        tubs.append(t)
cnt = 0
for t in tubs:
    cnt += len(t)
cnt

In [ ]:
dfs = []
for tub in tubs:
    df = pd.DataFrame(tub)
    name = Path(tub.base_path).name
    pref = os.path.join(name, Tub.images() ) + "/"
    df["cam/image_array"] = pref + df["cam/image_array"]
    dfs.append(df)
len(dfs)

In [ ]:
df_all = pd.concat(dfs)

In [ ]:
df_all.describe()

In [ ]:
tfms = [Normalize.from_stats([0.0692], [0.2051]),  # Normalize the images with the specified mean and standard deviation\n",
        *aug_transforms(do_flip=False, size=128)]  # Add default transformations except for horizontal flip\n",
tfms = [*aug_transforms(do_flip=False, size=128)]  # Add default transformations except for horizontal flip\n",
tfms = [Resize(128,method="squish")]
# Add to DataBlock: batch_tfms=tfms"

In [ ]:
pascal = DataBlock(blocks=(ImageBlock, RegressionBlock(n_out=2)),
                   splitter=RandomSplitter(),
                   get_x=ColReader("cam/image_array", pref=root_path),
                   get_y=ColReader(['user/angle','user/throttle']),
                   item_tfms=tfms,
                   n_inp=1)

In [ ]:
dls = pascal.dataloaders(df_all)
dls.show_batch()
dls.one_batch()[0].shape

In [ ]:
learn2 = cnn_learner(dls, resnet18, loss_func=MSELossFlat(), metrics=[rmse], cbs=ActivationStats(with_hist=True))
learn2.fine_tune(5)

In [ ]:
learn2.recorder.plot_loss()
learn2.show_results(figsize=(20,10))

The below code is modified from: https://github.com/cmasenas/fastai_navigation_training/blob/master/fastai_train.ipynb.
TODO: Figure out how to have multiple output heads

In [ ]:
model = torch.nn.Sequential(
        ConvLayer(3, 24, stride=2),
        ConvLayer(24, 32, stride=2),
        ConvLayer(32, 64, stride=2),
        ConvLayer(64, 128, stride=2),
        ConvLayer(128, 256, stride=2),
        nn.AdaptiveAvgPool2d(1),
        Flatten(),
        nn.Linear(256, 50),
        nn.Linear(50, dls.c)
        )
#print(model)

In [ ]:
#print(list(model.children()))
learn = Learner(dls, model,  loss_func = MSELossFlat(), metrics=[rmse], cbs=ActivationStats(with_hist=True))
valley = learn.lr_find()
print(valley)

In [ ]:
learn.fit_one_cycle(10, 1e-3)

In [ ]:
learn.recorder.plot_loss()
learn.show_results(figsize=(20,10))

In [ ]:
#dls=nav.dataloaders(df, bs=512)
preds, tgt = learn.get_preds(dl=[dls.one_batch()])

In [ ]:
plt.title("Target vs Predicted Steering", fontsize=18, y=1.0)
plt.xlabel("Target", fontsize=14, labelpad=15)
plt.ylabel("Predicted", fontsize=14, labelpad=15)
plt.plot(tgt.T[0], preds.T[0],'bo')
plt.plot([-1,1],[-1,1],'r', linewidth = 4)
plt.show()

In [ ]:
plt.title("Target vs Predicted Throttle", fontsize=18, y=1.02)
plt.xlabel("Target", fontsize=14, labelpad=15)
plt.ylabel("Predicted", fontsize=14, labelpad=15)
plt.plot(tgt.T[1], preds.T[1],'bo')
plt.plot([0,1],[0,1],'r', linewidth = 4)
plt.show()

In [ ]:
learn.export()